In [1]:
CV_PATH = "../data/people_resume.pdf"

In [2]:
from pdfminer.high_level import extract_text
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

# api key setup
os.environ["GORQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
cv_data = extract_text(CV_PATH)

In [5]:
cv_data

'Aashutosh Rathi\n\naashutosh.dev\n\nLast Updated on 6th January 2025\n\nEDUCATION\nIIIT VADODARA\nB.TECH IN COMPUTER SCIENCE\n2016-2020 | Gandhinagar, India\nCPI: 8.92/10\n\nOPENCLASSROOM\nFRONTEND DEVELOPER PATH\nJan 2019 - Jan 2020\n\nLINKS\nWebsite:// aashutosh.dev\nGitHub:// @aashutoshrathi\nNewsletter:// The Nibbles\nBlog:// xp.tar.gz\nEmail:// aashutoshrathi@gmail.com\n\nCOURSEWORK\nUNDERGRADUATE\nAlgorithms\nOperating Systems\nData Structures\nComputer Networks\nDistributed Systems\nProbability and Statistics\nArtiﬁcial Intelligence\n\nSKILLS\nPROGRAMMING\nAdvanced:\nJavaScript • Python • Java\nC++ • AWS • Shell • CSS\nDart • Go • SQL\nIntermediate:\nSolidity • PHP\nAndroid • Rust\n\nEXPERIENCE\nLEAD SOFTWARE ENGINEER STACKR LABS\nFeb 2024 - Dec 2024\nTech Stack: TypeScript, Go, Rust, AWS, Solidity, Wasm\n\n• Worked on SDK for building rollups like regular web applications and auxiliary\n\ntools\n\n• Led the on-chain projects dChess, Chess rollup and Comets, Atari’s game.\n• Le

In [6]:
## LLM setup

llm = ChatGroq(
    model="openai/gpt-oss-120b"
)

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [8]:
template = PromptTemplate(
    template = """
        You are a resume parsing assistant.

        Your job is to extract structured information from a resume and return the data in valid JSON format. 
        Only return the JSON object — do not include any explanations or extra text.

        Extract the following fields:

        - full_name
        - email
        - skills (as a list of strings)
        - education (as a list of objects with degree, institution, and optionally year)
        - work_experience (as a list of objects with job_title, company, and optionally years)

        If a field is not found, use `null` or an empty list where appropriate.

        Resume text: 
        \n\n\n
        {resume_context}
        \n\n\n

        Return only the parsed JSON below:
        """,
    input_variables=["resume_context"]
)

In [9]:
parser = JsonOutputParser()

In [10]:
chain = template | llm | parser

In [11]:
resume_extracted_output = chain.invoke({
    "resume_context" : cv_data
})

In [12]:
# Save cv data
import json

# save path
json_save_path = "../data/parsed_resume.json"

with open(json_save_path, "w") as file:
    json.dump(resume_extracted_output, file, indent=2)

In [13]:
## load parsed data

with open(json_save_path, "r") as file:
    parsed_resume = json.load(file)


In [16]:
### match job description

from langchain.prompts import PromptTemplate

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["job_description", "resume"],
    template="""
        You are an AI recruitment assistant.

        Your task is to evaluate how well a candidate’s resume aligns with the given job description.

        ### Instructions:
        1. Carefully analyze the **Job Description** and the **Resume** provided.
        2. Evaluate the candidate’s skills, experience, and qualifications against the job requirements.
        3. Provide:
        - A **match percentage score** (only the number with a % sign, e.g., "85%").

        ### Input Data:
        Job Description:
        {job_description}

        Resume:
        {resume}

        ### Output Format (JSON only):
        {{
        "score": "<percentage, e.g., 85%>"
        }}
        """
)



In [17]:
final_chain = prompt | llm | parser

In [32]:
job_des = """
    We are looking for a UI/UX Designer with experience in Figma, Adobe XD, and responsive design principles. The candidate should be able to create wireframes, prototypes, and collaborate closely with developers.
"""

In [33]:
result = final_chain.invoke({
    "resume" : parsed_resume,
    "job_description" : job_des
})

In [34]:
result
score = int(result["score"].replace('%',''))

In [35]:
score

10

In [41]:
if score > 60:
    print("Score is fine")
else:
    rewrite_cv = PromptTemplate(
        input_variables=["resume", "job_description"],
        template="""
    You are an AI assistant. The candidate's current resume may not fully align with the given job description.

    ### Instructions:
    1. Rewrite the resume in JSON format to better highlight the skills, experience, and qualifications relevant to the job description.
    2. Preserve the candidate's original information but reorder, emphasize, or rephrase it to match the job requirements.
    3. Return the rewritten resume strictly in JSON format. Do not include any extra text.

    ### Input:
    Resume:
    {resume}

    Job Description:
    {job_description}

    ### Output:
    JSON only
    """
    )
    
    
    chain = rewrite_cv | llm | parser
    output_result = chain.invoke({
        "resume" : parsed_resume,
        "job_description" : job_des
    })
    
    
    

In [42]:
output_result

{'full_name': 'Aashutosh Rathi',
 'email': 'aashutoshrathi@gmail.com',
 'summary': 'Software engineer with strong front‑end development expertise and a focus on creating responsive, user‑centric interfaces. Experienced in translating wireframes and prototypes into production‑ready UI using JavaScript, CSS, and modern frameworks, and collaborating closely with design and development teams.',
 'skills': ['CSS',
  'JavaScript',
  'Dart',
  'Flutter',
  'HTML',
  'Responsive Design',
  'Wireframing',
  'Prototyping',
  'Python',
  'Java',
  'C++',
  'Go',
  'SQL',
  'AWS',
  'Shell',
  'Solidity',
  'PHP',
  'Android',
  'Rust'],
 'education': [{'degree': 'B.TECH IN COMPUTER SCIENCE',
   'institution': 'IIIT VADODARA',
   'year': '2016-2020'},
  {'degree': 'FRONTEND DEVELOPER PATH',
   'institution': 'OPENCLASSROOM',
   'year': 'Jan 2019 - Jan 2020'}],
 'work_experience': [{'job_title': 'LEAD SOFTWARE ENGINEER',
   'company': 'STACKR LABS',
   'years': 'Feb 2024 - Dec 2024',
   'responsibi

In [43]:
result = final_chain.invoke({
    "resume" : output_result,
    "job_description" : job_des
})

In [44]:
result

{'score': '72%'}